<a href="https://colab.research.google.com/github/prostodema/PRACTIC/blob/prostodema-patch-1/HW_Dementiev3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

In [8]:
from google.colab import drive
drive.mount('/content/drive')

#from google.colab import files
#files.upload()

Mounted at /content/drive


In [9]:
file_path_data = '/content/drive/My Drive/Colab Notebooks/survey_results_public.csv'
file_path_schema = '/content/drive/My Drive/Colab Notebooks/survey_results_schema.csv'


In [10]:
survey_data = pd.read_csv(file_path_data)
schema_data = pd.read_csv(file_path_schema)

#survey_data.head()
#schema_data.head()


In [11]:
#Display the first few rows of the dataset to understand its structure
survey_data.head()
schema_data.head()

,qid,qname,question,force_resp,type,selector
0,QID2,MainBranch,Which of the following options best describes ...,True,MC,SAVR
1,QID127,Age,What is your age?*,True,MC,SAVR
2,QID296,Employment,Which of the following best describes your cur...,True,MC,MAVR
3,QID308,RemoteWork,Which best describes your current work situation?,False,MC,SAVR
4,QID341,Check,Just checking to make sure you are paying atte...,True,MC,SAVR


In [12]:
# 1: How many respondents took the survey?
true_columns = schema_data[schema_data['force_resp'] == True]['qname'].tolist()
filtered_data = survey_data[true_columns]
complete_responses = filtered_data.dropna()

Кількість респондентів: 65437


In [17]:
print(f"Кількість респондентів, які відповіли на всі питання: {complete_responses.shape[0]}")

Кількість респондентів, які відповіли на всі питання: 42806


In [13]:
# 3: What are the central tendency measures (mean, median, mode) for respondents' work experience (WorkExp)?
work_exp_data = pd.to_numeric(survey_data['WorkExp'], errors='coerce')

mean_exp = work_exp_data.mean()
median_exp = work_exp_data.median()
mode_exp = work_exp_data.mode()[0]

print(f'Mean: {mean_exp}, Median: {median_exp}, Moda: {mode_exp}')

Mean: 11.46695663901814, Median: 9.0, Moda: 3.0


In [14]:
# 4: How many respondents work remotely?
remote_workers = survey_data[survey_data['RemoteWork'] == 'Remote'].shape[0]
print(f'Кількість респондентів, які працюють віддалено: {remote_workers}')

Кількість респондентів, які працюють віддалено: 20831


In [15]:
# 5: What percentage of respondents program in Python?
python_programmers = survey_data[survey_data['LanguageHaveWorkedWith'].str.contains('Python', na=False)].shape[0]
total_respondents_non_null = survey_data['LanguageHaveWorkedWith'].notnull().sum()
python_percentage = (python_programmers / total_respondents_non_null) * 100

print(f'Відсоток програмістів на Python: {python_percentage:.2f}%')


Відсоток програмістів на Python: 51.54%


In [16]:
# 6: How many respondents learned programming through online courses?
online_learners = survey_data[survey_data['LearnCode'].str.contains('Online', na=False)].shape[0]
print(f'Кількість респондентів, які навчалися через онлайн курси: {online_learners}')


Кількість респондентів, які навчалися через онлайн курси: 30271


In [18]:
# 7: Among respondents who program in Python, grouped by country, what are the average and median annual compensation (ConvertedCompYearly) in each country?
python_users = survey_data[survey_data['LanguageHaveWorkedWith'].str.contains('Python', na=False)]

#filter
compensation_stats = python_users.groupby('Country')['ConvertedCompYearly'].agg(['mean', 'median'])
print(compensation_stats)

                                               mean    median
Country                                                      
Afghanistan                             4543.000000    4768.5
Albania                                56295.000000   56295.0
Algeria                                 9053.285714    6230.0
Andorra                               193331.000000  193331.0
Angola                                     6.000000       6.0
...                                             ...       ...
Venezuela, Bolivarian Republic of...   21500.000000    7100.0
Viet Nam                               14014.562500   10180.0
Yemen                                  10297.333333    5333.0
Zambia                                 28123.666667   22803.0
Zimbabwe                               37500.000000   18000.0

[173 rows x 2 columns]


In [19]:
# 8: What are the education levels of the 5 respondents with the highest compensation?
top_5_education = survey_data.nlargest(5, 'ConvertedCompYearly')[['EdLevel', 'ConvertedCompYearly']]
print(top_5_education)

                                              EdLevel  ConvertedCompYearly
15837    Bachelor’s degree (B.A., B.S., B.Eng., etc.)           16256603.0
12723  Professional degree (JD, MD, Ph.D, Ed.D, etc.)           13818022.0
28379  Professional degree (JD, MD, Ph.D, Ed.D, etc.)            9000000.0
17593    Bachelor’s degree (B.A., B.S., B.Eng., etc.)            6340564.0
17672  Professional degree (JD, MD, Ph.D, Ed.D, etc.)            4936778.0


In [20]:
# Bonus 1: In each age category, what percentage of respondents program in Python?
age_groups = survey_data.groupby('Age')['LanguageHaveWorkedWith'].apply(lambda x: (x.str.contains('Python', na=False).mean()) * 100)
print(age_groups)

Age
18-24 years old       55.922826
25-34 years old       45.773912
35-44 years old       41.520546
45-54 years old       41.910706
55-64 years old       40.427184
65 years or older     37.564767
Prefer not to say     45.341615
Under 18 years old    64.875389
Name: LanguageHaveWorkedWith, dtype: float64


In [24]:
# Bonus 2.1: 75 percentile
high_earning_remote = survey_data[(survey_data['ConvertedCompYearly'] >= survey_data['ConvertedCompYearly'].quantile(0.75)) &
                                  (survey_data['RemoteWork'] == 'Remote')]

In [22]:
# Bonus 2.2: Respondents in the 75th percentile that has compensation
industry_distribution = high_earning_remote['Employment'].value_counts()
print(industry_distribution)

Employment
Employed, full-time                                                                                                               2303
Independent contractor, freelancer, or self-employed                                                                               362
Employed, full-time;Independent contractor, freelancer, or self-employed                                                           296
Employed, full-time;Student, part-time                                                                                              21
Not employed, but looking for work;Independent contractor, freelancer, or self-employed                                             16
Independent contractor, freelancer, or self-employed;Employed, part-time                                                            15
Employed, part-time                                                                                                                 11
Employed, full-time;Independent contractor, 